In [0]:
# %run ../00_config

In [0]:
print("コンテンツマスタのCSVファイルを作成します...")

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col
import random

# ============================================================
# テーブル: ife_contents（機内エンタメコンテンツマスタ）サンプル
# ============================================================

# ---------- 設定 ----------
# GitHubイメージベースURL（仮：後であなたが修正）
git_owner  = "komae5519pv"
git_repo   = "komae_dbdemos"
git_img_dir = "/airline_recommends_20250429/_images"

def make_img_url(file_name: str) -> str:
    """画像URLを組み立て"""
    return f"https://github.com/{git_owner}/{git_repo}/blob/main{git_img_dir}/{file_name}?raw=true"

# ---------- 固定リスト ----------
contents = [
    # 映画、ドラマ (1〜20)
    {"content_id": i, "content_category": "映画、ドラマ", "content_img_url": make_img_url(f"{i}.png")}
    for i in range(1, 21)
] + [
    # 観光情報 (21〜22)
    {"content_id": 21, "content_category": "観光情報", "content_img_url": make_img_url("21.png")},
    {"content_id": 22, "content_category": "観光情報", "content_img_url": make_img_url("22.png")},
    # 食事・グルメ情報 (23〜24)
    {"content_id": 23, "content_category": "食事・グルメ情報", "content_img_url": make_img_url("23.png")},
    {"content_id": 24, "content_category": "食事・グルメ情報", "content_img_url": make_img_url("24.png")},
    # 移送手段や便利な情報 (25〜26)
    {"content_id": 25, "content_category": "移送手段や便利な情報", "content_img_url": make_img_url("25.png")},
    {"content_id": 26, "content_category": "移送手段や便利な情報", "content_img_url": make_img_url("26.png")},
    # リラックスできるコンテンツ (27〜28)
    {"content_id": 27, "content_category": "リラックスできるコンテンツ", "content_img_url": make_img_url("27.png")},
    {"content_id": 28, "content_category": "リラックスできるコンテンツ", "content_img_url": make_img_url("28.png")},
    # 文化やマナーに関する情報 (29〜30)
    {"content_id": 29, "content_category": "文化やマナーに関する情報", "content_img_url": make_img_url("29.png")},
    {"content_id": 30, "content_category": "文化やマナーに関する情報", "content_img_url": make_img_url("30.png")}
]

# ---------- 行レコードを組み立て ----------
rows = []
for content in contents:
    # duration_secはカテゴリごとにそれっぽい乱数を割り当て
    if content["content_category"] == "映画、ドラマ":
        duration_sec = random.randint(90*60, 120*60)  # 90〜120分
    else:
        duration_sec = random.randint(5*60, 15*60)    # 5〜15分

    rows.append(
        Row(
            content_id       = content["content_id"],
            content_category = content["content_category"],
            duration_sec     = duration_sec,
            content_img_url  = content["content_img_url"]
        )
    )

# ---------- DataFrame 化 ----------
df = spark.createDataFrame(rows)

# ---------- 動作確認 ----------
print("コンテンツレコード数:", df.count())      # 例: 30本
# df.show(10, truncate=False)
df.printSchema()
# print("カテゴリごとの作品数")
# df.groupBy("content_category").count().orderBy("content_category").show(truncate=False)

# # ---------- Delta テーブル保存 ----------
# df.write.format("delta")\
#   .option("comment", "機内エンタメコンテンツマスタ")\
#   .mode("overwrite")\
#   .saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_ife_contents")

# ---------- CSV 出力 ----------
out_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/ife_contents/ife_contents.csv"
df.coalesce(1).toPandas().to_csv(out_path, index=False)
print(out_path)

In [0]:
print("コンテンツマスタのCSVファイルを作成しました！")